In [1]:
import json

import matplotlib.pyplot as plt
import numpy as np
import vedo

from histalign.backend.io import load_image
from histalign.backend.models import AlignmentSettings
from histalign.backend.registration import Registrator

vedo.settings.default_backend = "vtk"

In [2]:
def show(volumes: list[vedo.CommonVisual], interactive: bool = True) -> None:
    vedo.Plotter().show(volumes, axes=3, interactive=interactive)


def imshow(image: np.ndarray) -> None:
    _, axes = plt.subplots()

    axes.imshow(image)
    axes.axis(False)

    plt.show()


def forward_register(
    image: np.ndarray,
    alignment_settings: AlignmentSettings,
) -> np.ndarray:
    registrator = Registrator(True, True)
    return registrator.get_forwarded_image(image, alignment_settings)

In [3]:
# alignment_path = "/home/ediun/git/histalign/projects/project_310724_full/7d9af388b0/7dff67f4735a5c6e74e99dfb959a38ed.json"
alignment_path = "/home/ediun/git/histalign/projects/project_temp4/12d4fcac65/7dff67f4735a5c6e74e99dfb959a38ed.json"

with open(alignment_path) as handle:
    alignment_settings = AlignmentSettings(**json.load(handle))

In [4]:
alignment_settings.histology_settings.rotation = 0

In [5]:
slice_ = load_image(alignment_settings.histology_path)

In [6]:
slice_ = forward_register(slice_, alignment_settings).T

image.shape=(316, 489), target_shape=(320, 528)
vertical_padding=4, horizontal_padding=39


In [7]:
slice_array = np.zeros(shape=alignment_settings.volume_settings.shape)

# match alignment_settings.volume_settings.orientation:
#     case "coronal":
#         raise ValueError("Panic")
#     case "horizontal":
#         raise ValueError("Panic")
#     case "sagittal":
#         slice_array[..., slice_array.shape[2] // 2 + alignment_settings.volume_settings.offset] = slice_

In [8]:
# slice_volume = vedo.Volume(np.zeros_like(slice_array))
slice_volume = vedo.Volume(slice_array)

In [9]:
pivot = slice_volume.center()
pivot[2] += alignment_settings.volume_settings.offset

# slice_volume.tonumpy()[..., int(pivot[2])] = slice_

# slice_volume.rotate_y(-alignment_settings.volume_settings.pitch, around=pivot)
# slice_volume.rotate_z(alignment_settings.volume_settings.yaw, around=pivot)

In [10]:
# show(slice_volume)

In [11]:
match alignment_settings.volume_settings.orientation:
    case "coronal":
        raise ValueError("Panic")
    case "horizontal":
        raise ValueError("Panic")
    case "sagittal":
        pivot = slice_volume.center()
        pivot[2] += alignment_settings.volume_settings.offset

        slice_volume.rotate_x(-alignment_settings.volume_settings.pitch)

        print(slice_volume.tonumpy().shape)

        slice_volume.tonumpy()[..., int(pivot[2])] = slice_

        slice_volume.rotate_y(alignment_settings.volume_settings.pitch)
        slice_volume.rotate_z(-alignment_settings.volume_settings.yaw)

(528, 328, 461)


ValueError: could not broadcast input array from shape (528,320) into shape (528,328)

In [ ]:
show(slice_volume)